In [ ]:
# ============================================================
# 🚀 Custom Voice Label Tool
# Author: Mr.Jack (https://github.com/Mr-Jack-Tung)
# Date: 2025-11-02
# Description: Công cụ ghi âm và gán nhãn giọng nói tùy chỉnh, tạo custom dataset với giao diện web sử dụng Gradio
# ============================================================

In [1]:
!pip install gradio soundfile datasets --quiet

In [2]:
import gradio as gr
import os, json, time, uuid, soundfile as sf
from datetime import datetime

/Users/mr.jack/Downloads/meeting-minutes/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# 🔧 Thư mục lưu dữ liệu
DATA_DIR = "voice_label_data"
os.makedirs(DATA_DIR, exist_ok=True)
META_FILE = os.path.join(DATA_DIR, "metadata.json")

# Tạo file metadata nếu chưa có
if not os.path.exists(META_FILE):
    with open(META_FILE, "w") as f:
        json.dump([], f)

# 📦 Hàm lưu audio + nhãn
def save_label(audio, transcript):
    if audio is None:
        return "⚠️ Bạn chưa ghi âm hoặc upload audio nào.", None

    # Audio có thể là dict {"name":..., "data":...}
    sr, wav = None, None
    if isinstance(audio, dict) and audio.get("data") is not None:
        sr, wav = audio["sampling_rate"], audio["data"]
    else:
        # fallback: gradio file object
        audio_path = audio if isinstance(audio, str) else audio.name
        wav, sr = sf.read(audio_path)

    # Lưu file wav mới
    filename = f"{datetime.now().strftime('%Y%m%d_%H%M%S')}_{uuid.uuid4().hex[:8]}.wav"
    wav_path = os.path.join(DATA_DIR, filename)
    sf.write(wav_path, wav, sr)

    # Lưu metadata
    entry = {"audio": wav_path, "transcription": transcript}
    with open(META_FILE, "r") as f:
        data = json.load(f)
    data.append(entry)
    with open(META_FILE, "w") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    return f"✅ Đã lưu nhãn cho file {filename}", gr.update(value="")

# 📜 Xem danh sách dữ liệu đã ghi nhãn
def list_labels():
    with open(META_FILE, "r") as f:
        data = json.load(f)
    if not data:
        return "Chưa có dữ liệu nào được gán nhãn."
    return "\n".join([f"{i+1}. {os.path.basename(d['audio'])} — {d['transcription']}" for i, d in enumerate(data)])

# 🧩 Giao diện Gradio
with gr.Blocks() as demo:
    gr.Markdown("# 🎙️ Voice Label Tool (for Whisper Fine-tuning)\nTạo dữ liệu `audio + transcription` dễ dàng trong Colab!")

    with gr.Row():
        with gr.Column():
            audio_input = gr.Audio(sources=["microphone", "upload"], type="filepath", label="🎤 Ghi âm hoặc tải audio")
            transcript = gr.Textbox(label="✍️ Nhập hoặc chỉnh sửa văn bản", lines=2)
            save_btn = gr.Button("💾 Lưu dữ liệu")
            msg = gr.Textbox(label="Thông báo", interactive=False)
            save_btn.click(save_label, inputs=[audio_input, transcript], outputs=[msg, transcript])

        with gr.Column():
            gr.Markdown("## 🧾 Dữ liệu đã ghi nhãn")
            refresh_btn = gr.Button("🔄 Làm mới danh sách")
            label_list = gr.Textbox(label="Danh sách nhãn", lines=12, interactive=False)
            refresh_btn.click(list_labels, outputs=label_list)

    gr.Markdown("👉 Tất cả dữ liệu được lưu tại `/content/voice_label_data/`.\nBạn có thể tải xuống và dùng để fine-tune Whisper!")

demo.launch(share=True, debug=True)

# Note: to recording audio in local, open the link with browser.


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://fe61ebb3451521d986.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://fe61ebb3451521d986.gradio.live


In [4]:
# 💾 Sau khi gán nhãn xong

# Tất cả dữ liệu của bạn sẽ được lưu trong:
# /content/voice_label_data/
# ├── audio_001.wav
# ├── audio_002.wav
# └── metadata.json


# File metadata.json sẽ có cấu trúc chuẩn Hugging Face:
# [
#   {"audio": "voice_label_data/audio_001.wav", "transcription": "xin chào bạn"},
#   {"audio": "voice_label_data/audio_002.wav", "transcription": "tôi là trợ lý AI"}
# ]


# Bạn có thể load lại rất dễ:
from datasets import load_dataset
dataset = load_dataset("json", data_files="voice_label_data/metadata.json")
dataset

Generating train split: 3 examples [00:00, 111.31 examples/s]


DatasetDict({
    train: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 3
    })
})